# NOTEBOOK DESAFIO 1 PNL
Alumna: Sofia Speri

## Código de la clase

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

### Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

### Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

## Desafío 1

#### **1.**
**Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.**


In [ ]:
#Vectorizador TF-IDF
#Filtro terminos que tienen más de 0.95 de frecuencia, que pueden ser términos que no aportan información
#Y sirvió para mejorar el f1_score también
tfidfvect_new = TfidfVectorizer(max_df = 0.95)

In [ ]:
#Obtengo la matriz documento-termino con el vectorizador
X_train_new = tfidfvect_new.fit_transform(newsgroups_train.data)

In [ ]:
#Target de entrenamiento
y_train_new = newsgroups_train.target

In [ ]:
#Indices de los 5 documentos que tomo al azar
idx = [9206, 5301, 1, 8385, 9201]

In [ ]:
#Muestro el texto y sus 5 mas similares según la similutd coseno
#Muestro los primeros 500 caracteres de cada documento en el print
for i in range(len(idx)):

  #Print del documento elegido y su clase
  print("_" * 100)
  print("_" * 100)
  print(f'Documento {idx[i]}')
  print("-" * 40)
  print(newsgroups_train.data[idx[i]][:500]+ " ...")
  print("-" * 40)
  print(f'Clase: {newsgroups_train.target_names[y_train_new[idx[i]]]}')

  #Medición de similaridad coseno con todos los documentos
  cossim = cosine_similarity(X_train_new[idx[i]], X_train_new)[0]

  # los 5 documentos más similares:
  mostsim = np.argsort(cossim)[::-1][1:6]

  #Muestro los 5 documentos más similares y sus clases
  print("-" * 40)
  print(f'Los 5 documentos más similares son:')
  for i_sim in mostsim:
    print("-" * 40)
    print("-" * 40)
    print(f'Documento {i_sim}')
    print("-" * 40)
    print(newsgroups_train.data[i_sim][:500]+ " ...")
    print("-" * 40)
    print(f'Similitud de coseno: {cossim[i_sim]:.4f}')
    print(f'Clase: {newsgroups_train.target_names[y_train_new[i_sim]]}')

____________________________________________________________________________________________________
____________________________________________________________________________________________________
Documento 9206
----------------------------------------
I have a spare Fluke 8062A.  This is a true RMS digital meter with
4.5 digit display.  It's in "average" condition (been used) but 
works fine - which is what Fluke's are all about.   BTW - according
to the fellow who designed these meters, they can be converted to 8060A
function by clipping a jumper, and adding the 2 extra switches..
Like I said $115 seems fair - these sell for $300+ new.

 ...
----------------------------------------
Clase: sci.electronics
----------------------------------------
Los 5 documentos más similares son:
----------------------------------------
----------------------------------------
Documento 948
----------------------------------------
I've had my Fluke 8060A here at work for just over 10 years now.


Como resultado vemos que para los 5 documentos tomados al azar que eran de diferentes clases entre sí, en todos los casos la similitud coseno funcionó correctamente para encontrar documentos que estuvieran en la misma clase que el documento a evaluar o en clases relativamente cercanas como por ejemplo las clases *sci.electronics* con *comp.sys.ibm.pc.hardware*

#### **2.**  
**Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.**


##### Modelo MultinomialNB

In [ ]:
clf_multi_new = MultinomialNB(alpha=0.01)
clf_multi_new.fit(X_train_new, y_train_new)

MultinomialNB(alpha=0.01)

In [ ]:
X_test_m = tfidfvect_new.transform(newsgroups_test.data)
y_test_m = newsgroups_test.target
y_pred_m =  clf_multi_new.predict(X_test_m)

In [ ]:
f1 = f1_score(y_test_m, y_pred_m, average='macro')
print(f'El valor de f1 score obtenido para MultinomialNB es {f1}')

El valor de f1 score obtenido para MultinomialNB es 0.682861129525057


##### Modelo ComplementNB

In [ ]:
clf_compl_new = ComplementNB(alpha=0.3)
clf_compl_new.fit(X_train_new, y_train_new)

ComplementNB(alpha=0.3)

In [ ]:
X_test_c = tfidfvect_new.transform(newsgroups_test.data)
y_test_c = newsgroups_test.target
y_pred_c =  clf_compl_new.predict(X_test_c)

In [ ]:
f1 = f1_score(y_test_c, y_pred_c, average='macro')
print(f'El valor de f1 score obtenido para MultinomialNB es {f1}')

El valor de f1 score obtenido para MultinomialNB es 0.6999368058272992


Se entrenron los dos modelos propuestos de NB, en ambos el hiperparámetro que mejoró el f1-score con el vectorizador propuesto, fue la variación del alpha. En ambos casos bajar el valor del alpha, es decir del smoothing para evitar que las clases sean 0. Eso puede deberse a que al tener suficientes datos de forma de que la probabildad de que en el set de entrenamiento se encuentren todas las clases es alta; con un bajo valor de alpha no altero o penalizo las probabilidades de cada carecterstica

#### **3**.
Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [ ]:
# Obtengo la matriz termino-documento como la traspuesta de la matriz documento-termino
mat_doc_ter = X_train_new.toarray()
mat_ter_doc = mat_doc_ter.T

In [ ]:
idx2word_new = {v: k for k,v in tfidfvect_new.vocabulary_.items()}

In [ ]:
# Palabras a evaluar
palabras = ['germany','religion','sport','computer','board']

for palabra in palabras:
  print("_" * 100)
  print(f'Palabra: {palabra}')
  if palabra in tfidfvect_new.vocabulary_:
    # Indice de la palabra
    indice = tfidfvect_new.vocabulary_[palabra]

    # vector de la matriz correspondiente a ese término
    vector_palabra = mat_ter_doc[indice,:].reshape(1,-1)

    #Similutd coseno de la palabra con las otros vectores de palabra
    cossim = cosine_similarity(vector_palabra, mat_ter_doc)[0]
    mostsim = np.argsort(cossim)[::-1][1:6]

    print('Las 5 palabras más similares son:')
    for i in mostsim:
      similar_palabra = idx2word_new[i]
      print(f"\tPalabra: {similar_palabra}, Similitud: {cossim[i]:.4f}")

  else:
    print(f"La palabra {palabra} no se encuentra en el vocabulario")

____________________________________________________________________________________________________
Palabra: germany
Las 5 palabras más similares son:
	Palabra: landshut, Similitud: 0.3093
	Palabra: hedos, Similitud: 0.3093
	Palabra: probul, Similitud: 0.2901
	Palabra: probulf, Similitud: 0.2901
	Palabra: emanuelstr, Similitud: 0.2901
____________________________________________________________________________________________________
Palabra: religion
Las 5 palabras más similares son:
	Palabra: religious, Similitud: 0.2451
	Palabra: religions, Similitud: 0.2116
	Palabra: categorized, Similitud: 0.2039
	Palabra: purpsoe, Similitud: 0.2008
	Palabra: crusades, Similitud: 0.1987
____________________________________________________________________________________________________
Palabra: sport
Las 5 palabras más similares son:
	Palabra: woops, Similitud: 0.4388
	Palabra: pigeonhole, Similitud: 0.2921
	Palabra: rec, Similitud: 0.2700
	Palabra: golf, Similitud: 0.2418
	Palabra: fj1200, Simil

En este caso veo que es más dificil que encuentre palabras similares salvo casos muy específicos como es el caso de "religion"; pero puede deberse a la amplia gama de documentos y temas, donde puede que esa palabra no haya tenido tanta frecuencia en los documentos y puede que un vector de una palabra se relacione a otros términos que hayan aparecido cerca y la similitud coseno no va a ser tan epecífica